Less of a unit test, more an exploration of waffles.

In [1]:
filename = 'samples/waffle.tex'

In [3]:
# debug
from importlib import reload
import TexSoup
reload(TexSoup)

import TexSoup
import re
import numpy as np

from TexSoup import preprocessing

In [4]:
with open(filename,'r') as f:
    tex_doc = f.read()

Sometimes there are weird errors like spaces in begins/ends -- fix these first:

In [5]:
# from TexSoup import preprocessing
# reload(preprocessing)

In [6]:
tex_doc = preprocessing.process_begin_end(tex_doc)

Get new commands and enviroments -- we need to these to check if there are `\begin` and `\end` in these, so first, get them!

In [7]:
newcommands, newenvironments = preprocessing.get_newcommands_and_newenvs(tex_doc,
                                                                         verbose=True)

Get the argument numbers for these -- will need this for search-replace:

In [8]:
args_newcommands = preprocessing.find_args_newcommands(newcommands,error_out=True, verbose=True)

In [9]:
args_newcommands

[['\\be', '\\newcommand{\\be}{\\begin{equation}}', 34, 68, 0],
 ['\\ee', '\\newcommand{\\ee}{\\end{equation}}', 69, 101, 0],
 ['\\bee',
  '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
  103,
  153,
  1],
 ['\\eee', '\\newcommand{\\eee}{\\end{equation}}', 154, 187, 0],
 ['\\beee',
  '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
  189,
  243,
  2],
 ['\\eeee', '\\newcommand{\\eeee}{\\end{equation}}', 244, 278, 0],
 ['\\etal ', '\\def \\etal {{\\em et al.}}', 281, 306, 0]]

In [10]:
args_newenvironments = preprocessing.find_args_newenvironments(newenvironments,
                                                               error_out=True, verbose=True)

In [11]:
args_newenvironments

[]

In [13]:
comments, find_replace, error = preprocessing.generate_find_replace_newcommands(args_newcommands, 
                                                               verbose=True,arg_type = 'newcommand')

In [14]:
find_replace

[('\\be', '\\begin{equation}', 0),
 ('\\ee', '\\end{equation}', 0),
 ('\\bee', '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}', 1),
 ('\\eee', '\\end{equation}', 0),
 ('\\beee', '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}', 2),
 ('\\eeee', '\\end{equation}', 0)]

Replace newcommands and environments:

In [15]:
tex_doc_nc, error, warnings = preprocessing.replace_newcommands_and_newenvironments(tex_doc, 
                                                                          args_newcommands,
                                                                          args_newenvironments,
                                                                          verbose=True)

args for: \newcommand{\bee}[1]{\begin{equation}\label{e:#1}}
{1: 'lenseq'}
args for: \newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2}
{1: 'lenseq2', 2: 'A(x)=5'}

\newcommand{\be}{\begin{equation}} gets commented
\newcommand{\ee}{\end{equation}} gets commented
\newcommand{\bee}[1]{\begin{equation}\label{e:#1}} gets commented
\newcommand{\eee}{\end{equation}} gets commented
\newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2} gets commented
\newcommand{\eeee}{\end{equation}} gets commented


In [23]:
soup = TexSoup.TexSoup(r''+tex_doc_nc, tolerance=0)

In [24]:
reload(TexSoup)
import TexSoup.postprocess as postprocess
reload(postprocess)

<module 'TexSoup.postprocess' from '/Users/jnaiman/TexSoup/TexSoup/postprocess.py'>

In [25]:
soup_clean = postprocess.clean_slash_commands(soup)

In [19]:
**HERE**

In [20]:
# for iss,s in enumerate(soup.all):
#     print('----------- soup element =', iss, '---------------')
#     print(type(s))
#     print(type(s.expr))
#     print(s)
#     print('----------------')

In [21]:
# combine \command, \ as just \command\
for iss,s in enumerate(soup.all):
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                if str(ss) == '\\ ': # just a lone bracket
                    if type(s.all[isss-1].expr) == TexSoup.data.TexCmd: # is a command, update it
                        soup.all[iss].all[isss-1].name += '\\ ' # add 
                        soup.all[iss].all[isss].delete() # delete this node

In [26]:
for iss,s in enumerate(soup_clean.all):
    print('----------- soup element =', iss, '---------------')
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                print('----------- sub-soup element =', isss, '---------------')
                #print(type(ss))
                print(type(ss.expr))
                print(ss)
                print('-----------------')
                if str(ss).strip() == '\\': # 
                    import sys; sys.exit()
        else:
            print(' ******** not in document *********** ')

----------- soup element = 0 ---------------
----------- soup element = 1 ---------------
----------- soup element = 2 ---------------
----------- soup element = 3 ---------------
----------- soup element = 4 ---------------
----------- soup element = 5 ---------------
----------- soup element = 6 ---------------
----------- soup element = 7 ---------------
----------- soup element = 8 ---------------
----------- soup element = 9 ---------------
----------- soup element = 10 ---------------
----------- soup element = 11 ---------------
----------- soup element = 12 ---------------
----------- soup element = 13 ---------------
----------- soup element = 14 ---------------
----------- soup element = 15 ---------------
----------- soup element = 16 ---------------
----------- soup element = 17 ---------------
----------- soup element = 18 ---------------
----------- soup element = 19 ---------------
----------- soup element = 20 ---------------
----------- soup element = 21 --------------

In [ ]:
s.all[isss-1].__str__()

In [ ]:
s.all[isss-1].name += '\\ '

In [ ]:
s.all[isss-1]

In [ ]:
s.all[isss].delete()

In [ ]:
s.all[isss-1]

In [ ]:
s.all[isss]

In [ ]:
# try a thing
from TexSoup.category import categorize
from TexSoup.tokens import tokenize
from TexSoup.utils import Token, Buffer, MixedBuffer, CharToLineOffset

from TexSoup.reader import read_args, SIGNATURES,MODE_NON_MATH


In [ ]:
buf = Buffer(tokenize(categorize('\\sect  \t    \n\t{wallawalla}')))

In [ ]:
name = next(buf)

In [ ]:
name

In [ ]:
token = Token('', buf.position)

In [ ]:
token

In [ ]:
n_required_args=-1
n_optional_args=-1
tolerance = 0

In [ ]:
if n_required_args < 0 and n_optional_args < 0:
    n_required_args, n_optional_args = SIGNATURES.get(name, (-1, -1))

In [ ]:
n_required_args, n_optional_args

In [ ]:
args = read_args(buf, n_required_args, n_optional_args,
                 tolerance=tolerance, mode=MODE_NON_MATH)


In [ ]:
args

In [ ]:
cmd = '\\etal\\'

In [ ]:
reload(TexSoup)

In [ ]:
buf = tokenize(categorize(r''+cmd))

In [ ]:
buf

In [ ]:
def read_item_from(string, skip=2):
    buf = tokenize(categorize(string))
    _ = buf.forward(skip)
    return TexSoup.reader.read_item(buf)

In [ ]:
read_item_from(r'hi')

In [ ]:
read_item_from(cmd)